In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# category variable
category = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']

# continuous variable
continuous = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 
              'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']

In [87]:
#features = ['cat1', 'cont0', 'cat2', 'cont11', 'cont13', 'cat8', 'cont8', 'cont1', 'cat9',
#            'cont9', 'cont5', 'cat3', 'cat0', 'cat6', 'cont3', 'cat5', 'cont4', 'cont2', 'cont12']

train_dataset = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
dataset = pd.concat([train_dataset, test_data])

# あきらかな外れ値は削除
# [166042]はあきらかな外れ値
outlier=[166042]
for x in outlier:
    dataset = dataset.loc[dataset['id'] != x, :]

# idとtargetは避難させておく
id = dataset['id']
target = dataset['target']
# 避難させたので遠慮なく削除
dataset = dataset.drop(columns=['id', 'target'])
# 相関が高いものだけを使用
# この手法は有効だったと思うが、Catboostでのスコアは一定以上伸びなかった。

# 重要度を確認するためにはLabelEncodingが有効
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
encoder = LabelEncoder()
scaler = RobustScaler()

for x in dataset.columns:
    # notebookみると、みんなLabelEncoder使っているなぁ
    if dataset[x].dtype == object:
        dataset[x] = encoder.fit_transform(dataset[x])
        #dataset = pd.get_dummies(dataset, columns=[x], drop_first=True)
        
dataset[continuous] = scaler.fit_transform(dataset.loc[:,continuous].values)

# データセットにidとtargetを元に戻して
dataset = pd.concat([id,dataset,target], axis=1)
# targetのあるなしでtrainとtestを分割
train = dataset.loc[dataset['target'].notnull(), :]
test  = dataset.loc[dataset['target'].isnull(), :]

X = train.drop(columns=['id', 'target'])
y = train['target']
X_prediction = test.drop(columns=['id', 'target'])
prediction_id = test.loc[:,'id']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor

from sklearn.ensemble import StackingRegressor

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

def cross_val(model):
    score = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=kfold)
    print(f'{type(model).__name__} score = : {score.mean():.8f}')


regressors = [LinearRegression(), Lasso(alpha=0.005),
              #RandomForestRegressor(n_estimators = 100, criterion='mae', random_state=0, bootstrap=True),
              lgb.LGBMRegressor(objective='regression', num_leaves=5, learning_rate=0.05, n_estimators=720, bagging_fraction=0.8, verbose=10),
              GradientBoostingRegressor(n_estimators = 3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='RMSE', random_state=0),
              xgb.XGBRegressor(colsample_bytree=0.463, gamma=0.0468, learning_rate=0.05, max_depth=3, min_child_weight=1.7817, n_estimators=2200, reg_alpha=0.4640, reg_lambda=0.8571, subsample=0.5213, silent=1, random_state=0, nthread=-1)
             ]

#for x in regressors:
#    cross_val(x)

In [ ]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
light_grid = {'max_depth': [3, 5, 8],
             'learning_rate': [0.001, 0.0001, 0.0001],
             'n_estimators': [10, 100, 200, 500],
              'num_leaves': [100, 300, 500]
             }
lgb_grid_search = GridSearchCV(lgb.LGBMRegressor(), light_grid, cv=kfold, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
lgb_grid_search.fit(X,y)
lgb_param = lgb_grid_search.best_params_
print(lgb_param)


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.metrics import r2_score, mean_squared_error

class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
    
    def fit(self, X, y, X_test, y_test):
        self.models_ = [clone(x) for x in self.models]

        for model in self.models_:
            # CatBoostに関しては、eval_setが必須のためここでエラーが発生する。
            # その場合はexceptでfit処理を行う
            print(f'************ RUNNING : {model} ************')
            try:
                model.fit(X, y)
            except:
                model.fit(X, y, eval_set=(X_test, y_test))
                
            y_pred = model.predict(X_test)
            print(f'{model} RMSE: {mean_squared_error(y_test, y_pred)}')
            
        return self
    
    def predict(self, X):
        predictions = np.column_stack(
            [model.predict(X) for model in self.models_]
            )
        return np.mean(predictions, axis=1)

    
averaged_models = AveragingModels(models = (cat, lasso, lgb_tuned))
averaged_models.fit(X_train, y_train, X_test, y_test)

# 予想してみる
from sklearn.metrics import mean_squared_error
y_pred = averaged_models.predict(X_test)

In [91]:
import tensorflow as tf
from tensorflow import keras

def rmse(y_true, y_pred):
    return tf.sqrt(tf.losses.mean_squared_error(y_true, y_pred))

### ***********************************************************###
model = tf.keras.Sequential()

#model.add(tf.keras.layers.Dense(1024, activation='relu', input_shape=(X.shape[1],)))
#model.add(tf.keras.layers.LeakyReLU())
#model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(0.5))

#model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu',input_shape=(X.shape[1],)))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

#model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

#model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dense(32, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))

#model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dense(32, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))

#model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(tf.keras.layers.LeakyReLU())

model.add(tf.keras.layers.Dense(units=1, activation='linear'))
### ***********************************************************###

optimizer = tf.keras.optimizers.Adam(lr=0.005, decay=5e-4)
model.compile(optimizer = optimizer, loss = 'mae', metrics = ['mse', 'mae'])

#checkpoint_name = 'Model/{epoch:03d}-{val_loss:.5f}.hdf5'
checkpoint_name = 'DNN_BestModel.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callback_list = [checkpoint]

history = model.fit(X_train, y_train, validation_split=0.2, epochs = 500, batch_size = 1024,
                    validation_data=(X_test, y_test), callbacks=callback_list)
y_pred = model.predict(X_test).reshape(-1)
RMSE = rmse(y_test, y_pred)
print(f'RMSE = {RMSE}')

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)
ax.set_ylim(0.6, 0.75)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Train', 'Test'], loc='upper right')
plt.savefig(fname='1024 neurons enable LearningRate.png')

Epoch 1/1000
188/188 [==============================] - 3s 7ms/step - loss: 2.6093 - mse: 10.9217 - mae: 2.4182 - val_loss: 1.1590 - val_mse: 1.5188 - val_mae: 1.0212

Epoch 00001: val_loss improved from inf to 1.15903, saving model to DNN_BestModel.hdf5
Epoch 2/1000
188/188 [==============================] - 1s 7ms/step - loss: 1.0651 - mse: 1.3750 - mae: 0.9448 - val_loss: 0.8273 - val_mse: 0.8061 - val_mae: 0.7495

Epoch 00002: val_loss improved from 1.15903 to 0.82727, saving model to DNN_BestModel.hdf5
Epoch 3/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.8652 - mse: 0.9475 - mae: 0.7969 - val_loss: 0.7949 - val_mse: 0.8181 - val_mae: 0.7481

Epoch 00003: val_loss improved from 0.82727 to 0.79490, saving model to DNN_BestModel.hdf5
Epoch 4/1000
188/188 [==============================] - 1s 7ms/step - loss: 0.7934 - mse: 0.8259 - mae: 0.7501 - val_loss: 0.7658 - val_mse: 0.7689 - val_mae: 0.7307

Epoch 00004: val_loss improved from 0.79490 to 0.76577, saving

188/188 [==============================] - 1s 6ms/step - loss: 0.7271 - mse: 0.7711 - mae: 0.7242 - val_loss: 0.7235 - val_mse: 0.7613 - val_mae: 0.7206

Epoch 00035: val_loss did not improve from 0.72148
Epoch 36/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7247 - mse: 0.7647 - mae: 0.7219 - val_loss: 0.7209 - val_mse: 0.7629 - val_mae: 0.7182

Epoch 00036: val_loss improved from 0.72148 to 0.72090, saving model to DNN_BestModel.hdf5
Epoch 37/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7264 - mse: 0.7663 - mae: 0.7236 - val_loss: 0.7217 - val_mse: 0.7616 - val_mae: 0.7190

Epoch 00037: val_loss did not improve from 0.72090
Epoch 38/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7252 - mse: 0.7668 - mae: 0.7225 - val_loss: 0.7255 - val_mse: 0.7590 - val_mae: 0.7228

Epoch 00038: val_loss did not improve from 0.72090
Epoch 39/1000
188/188 [==============================] - 1s 7ms/step - loss: 0.7245 - mse: 0.7659 

188/188 [==============================] - 1s 6ms/step - loss: 0.7229 - mse: 0.7655 - mae: 0.7211 - val_loss: 0.7198 - val_mse: 0.7518 - val_mae: 0.7181

Epoch 00071: val_loss did not improve from 0.71888
Epoch 72/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7233 - mse: 0.7644 - mae: 0.7216 - val_loss: 0.7188 - val_mse: 0.7583 - val_mae: 0.7170

Epoch 00072: val_loss improved from 0.71888 to 0.71881, saving model to DNN_BestModel.hdf5
Epoch 73/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7238 - mse: 0.7671 - mae: 0.7221 - val_loss: 0.7187 - val_mse: 0.7571 - val_mae: 0.7170

Epoch 00073: val_loss improved from 0.71881 to 0.71873, saving model to DNN_BestModel.hdf5
Epoch 74/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7219 - mse: 0.7627 - mae: 0.7202 - val_loss: 0.7194 - val_mse: 0.7574 - val_mae: 0.7176

Epoch 00074: val_loss did not improve from 0.71873
Epoch 75/1000
188/188 [==============================] - 1


Epoch 00107: val_loss did not improve from 0.71847
Epoch 108/1000
188/188 [==============================] - 1s 7ms/step - loss: 0.7215 - mse: 0.7617 - mae: 0.7201 - val_loss: 0.7201 - val_mse: 0.7585 - val_mae: 0.7187

Epoch 00108: val_loss did not improve from 0.71847
Epoch 109/1000
188/188 [==============================] - 1s 7ms/step - loss: 0.7213 - mse: 0.7637 - mae: 0.7198 - val_loss: 0.7197 - val_mse: 0.7560 - val_mae: 0.7182

Epoch 00109: val_loss did not improve from 0.71847
Epoch 110/1000
188/188 [==============================] - 1s 7ms/step - loss: 0.7201 - mse: 0.7608 - mae: 0.7187 - val_loss: 0.7196 - val_mse: 0.7545 - val_mae: 0.7181

Epoch 00110: val_loss did not improve from 0.71847
Epoch 111/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7212 - mse: 0.7640 - mae: 0.7197 - val_loss: 0.7184 - val_mse: 0.7578 - val_mae: 0.7170

Epoch 00111: val_loss improved from 0.71847 to 0.71844, saving model to DNN_BestModel.hdf5
Epoch 112/1000
188/188 [====

188/188 [==============================] - 1s 6ms/step - loss: 0.7207 - mse: 0.7632 - mae: 0.7194 - val_loss: 0.7194 - val_mse: 0.7591 - val_mae: 0.7181

Epoch 00144: val_loss did not improve from 0.71820
Epoch 145/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7193 - mse: 0.7594 - mae: 0.7180 - val_loss: 0.7200 - val_mse: 0.7577 - val_mae: 0.7187

Epoch 00145: val_loss did not improve from 0.71820
Epoch 146/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7216 - mse: 0.7653 - mae: 0.7203 - val_loss: 0.7187 - val_mse: 0.7545 - val_mae: 0.7174

Epoch 00146: val_loss did not improve from 0.71820
Epoch 147/1000
188/188 [==============================] - 1s 7ms/step - loss: 0.7220 - mse: 0.7647 - mae: 0.7206 - val_loss: 0.7187 - val_mse: 0.7542 - val_mae: 0.7174

Epoch 00147: val_loss did not improve from 0.71820
Epoch 148/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7235 - mse: 0.7667 - mae: 0.7222 - val_loss: 0.7188 - v

...

188/188 [==============================] - 1s 6ms/step - loss: 0.7194 - mse: 0.7633 - mae: 0.7186 - val_loss: 0.7168 - val_mse: 0.7535 - val_mae: 0.7159

Epoch 00513: val_loss did not improve from 0.71650
Epoch 514/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7183 - mse: 0.7588 - mae: 0.7175 - val_loss: 0.7169 - val_mse: 0.7571 - val_mae: 0.7160

Epoch 00514: val_loss did not improve from 0.71650
Epoch 515/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.7190 - mse: 0.7616 - mae: 0.7181 - val_loss: 0.7170 - val_mse: 0.7602 - val_mae: 0.7162

Epoch 00515: val_loss did not improve from 0.71650
Epoch 516/1000
188/188 [==============================] - 1s 7ms/step - loss: 0.7176 - mse: 0.7594 - mae: 0.7168 - val_loss: 0.7174 - val_mse: 0.7551 - val_mae: 0.7166

Epoch 00516: val_loss did not improve from 0.71650
Epoch 517/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7187 - mse: 0.7597 - mae: 0.7179 - val_loss: 0.7172 - v

188/188 [==============================] - 1s 6ms/step - loss: 0.7183 - mse: 0.7600 - mae: 0.7175 - val_loss: 0.7167 - val_mse: 0.7562 - val_mae: 0.7159

Epoch 00550: val_loss did not improve from 0.71650
Epoch 551/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7187 - mse: 0.7611 - mae: 0.7179 - val_loss: 0.7166 - val_mse: 0.7547 - val_mae: 0.7158

Epoch 00551: val_loss did not improve from 0.71650
Epoch 552/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7200 - mse: 0.7632 - mae: 0.7192 - val_loss: 0.7173 - val_mse: 0.7591 - val_mae: 0.7164

Epoch 00552: val_loss did not improve from 0.71650
Epoch 553/1000
188/188 [==============================] - 1s 7ms/step - loss: 0.7203 - mse: 0.7656 - mae: 0.7195 - val_loss: 0.7172 - val_mse: 0.7564 - val_mae: 0.7164

Epoch 00553: val_loss did not improve from 0.71650
Epoch 554/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7174 - mse: 0.7590 - mae: 0.7165 - val_loss: 0.7167 - v

188/188 [==============================] - 1s 6ms/step - loss: 0.7187 - mse: 0.7589 - mae: 0.7179 - val_loss: 0.7172 - val_mse: 0.7541 - val_mae: 0.7164

Epoch 00587: val_loss did not improve from 0.71650
Epoch 588/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7191 - mse: 0.7621 - mae: 0.7183 - val_loss: 0.7170 - val_mse: 0.7565 - val_mae: 0.7162

Epoch 00588: val_loss did not improve from 0.71650
Epoch 589/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7180 - mse: 0.7586 - mae: 0.7172 - val_loss: 0.7168 - val_mse: 0.7565 - val_mae: 0.7160

Epoch 00589: val_loss did not improve from 0.71650
Epoch 590/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7190 - mse: 0.7616 - mae: 0.7182 - val_loss: 0.7166 - val_mse: 0.7565 - val_mae: 0.7158

Epoch 00590: val_loss did not improve from 0.71650
Epoch 591/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7172 - mse: 0.7588 - mae: 0.7164 - val_loss: 0.7166 - v

188/188 [==============================] - 1s 6ms/step - loss: 0.7184 - mse: 0.7589 - mae: 0.7176 - val_loss: 0.7171 - val_mse: 0.7570 - val_mae: 0.7163

Epoch 00624: val_loss did not improve from 0.71650
Epoch 625/1000
188/188 [==============================] - 1s 7ms/step - loss: 0.7183 - mse: 0.7588 - mae: 0.7175 - val_loss: 0.7169 - val_mse: 0.7560 - val_mae: 0.7161

Epoch 00625: val_loss did not improve from 0.71650
Epoch 626/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7185 - mse: 0.7605 - mae: 0.7177 - val_loss: 0.7172 - val_mse: 0.7552 - val_mae: 0.7165

Epoch 00626: val_loss did not improve from 0.71650
Epoch 627/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7184 - mse: 0.7619 - mae: 0.7177 - val_loss: 0.7170 - val_mse: 0.7550 - val_mae: 0.7162

Epoch 00627: val_loss did not improve from 0.71650
Epoch 628/1000
188/188 [==============================] - 1s 6ms/step - loss: 0.7167 - mse: 0.7568 - mae: 0.7159 - val_loss: 0.7169 - v

KeyboardInterrupt: 

In [ ]:
# CatBoosting
'''
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.model_selection import StratifiedKFold, KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

from sklearn.model_selection import GridSearchCV
param_cat = {'depth':[6,8,10],
            'learning_rate':[0.005, 0.001],
            'l2_leaf_reg':[1,4,9],
            'iterations':[100],
            'cat_features':[feature_cat],
            'eval_metric':['RMSE']
            }

grid_result = GridSearchCV(estimator=CatBoostRegressor(),param_grid=param_cat, cv=kfold, scoring='neg_mean_squared_error', n_jobs = -1, verbose=2)
grid_result.fit(X_train, y_train)
grid_param = grid_result.best_params_
print(grid_param)


cat = CatBoostRegressor(task_type='GPU', iterations=8000, use_best_model=True, depth=10, eval_metric='RMSE', l2_leaf_reg=1, learning_rate=0.001, early_stopping_rounds=10)
cat.fit(X_train, y_train, cat_features=feature_cat, eval_set = (X_test, y_test))
y_pred = cat.predict(X_test)
print(f'RMSE: {mean_squared_error(y_test, y_pred)}')


'''

In [ ]:
#y_pred=cat.predict(X_prediction)
output = pd.DataFrame({'id': prediction_id, 'target': y_pred})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")